---

# <center>DenseNet: Desvendando a Eficiência e Inovação em Redes Neurais Convolucionais (<strong style="color:green;">81,68%</strong>)</center>

---

## 1. Configurações de Ambiente

<p style="font-family: Arial, sans-serif; font-size: 16px; line-height: 1.5; color: #333; text-align: justify;">Antes de começarmos a construir e treinar nossa rede neural, precisamos configurar nosso ambiente. Isso envolve várias etapas importantes que garantem que nosso código seja executado corretamente e nossos experimentos sejam reproduzíveis.</p>

### 1.1 Instalação e Carga de Pacotes

In [1]:
!pip install lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 53.4 MB/s eta 0:00:00


In [2]:
# Ambiente de desenvolvimento
import os
import warnings

# Álgebra Linear & Math
import numpy as np
import pandas as pd

# Visualização de dados
import seaborn as sns
import matplotlib.pyplot as plt

# Pytorch
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR

# Métricas de Avaliação
from torchmetrics.classification import Accuracy

# Lightning
import lightning as pl
from lightning.pytorch import Trainer
from lightning import LightningModule
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import EarlyStopping
from lightning.pytorch.callbacks import ModelCheckpoint

# Torchvision
import torchvision
from torchvision.datasets import CIFAR100
from torchvision.transforms import Resize
from torchvision.models import densenet121
from torchvision.transforms import Compose
from torchvision.transforms import ToTensor
from torchvision.transforms import Normalize
from torchvision.transforms import ColorJitter
from torchvision.transforms import RandomRotation
from torchvision.transforms import RandomHorizontalFlip

In [3]:
# Ignorando avisos desnecessários
warnings.filterwarnings("ignore")

### 1.2 Garantindo a Reprodutibilidade dos Experimentos

In [4]:
def set_seed(seed=1996):
    """
    Configura a semente para geração de números aleatórios em várias bibliotecas para garantir a reprodutibilidade.
    :param seed: Valor da semente.
    """
    # Semente para NumPy
    np.random.seed(seed)

    # Semente para PyTorch
    torch.manual_seed(seed)

    # Semente para PyTorch Lightning
    pl.seed_everything(seed)

    # Semente para ambiente Python (Hashing)
    os.environ["PYTHONHASHSEED"] = str(seed)

    # Semente e configurações para GPU, se disponível
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# Chamada da função para inicializar as sementes
set_seed()

INFO: Seed set to 1996
INFO:lightning.fabric.utilities.seed:Seed set to 1996


### 1.3 Definindo o dispositivo (CPU OU GPU)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"O dispositivo usado: {device}")

O dispositivo usado: cuda


## 2. Carga e Pré-processamento das Imagens

### 2.1 Obtendo os DataLoaders

<ol>
    <li><p style="font-family: 'Times New Roman', Times, serif; font-size: 17px; text-align: justify;"><b>Média e Desvio Padrão:</b> O objetivo é criar uma classe Python que calcule a média e o desvio padrão das imagens originais. Esses valores são importantes porque serão usados posteriormente para padronizar as imagens.</p></li>

<li><p style="font-family: 'Times New Roman', Times, serif; font-size: 17px; text-align: justify;"><b>Criação dos Transformadores:</b> Após calcular a média e o desvio padrão, o próximo passo é criar os transformadores. Estamos utilizando o módulo <code>torchvision.transforms</code> para isso. Os transformadores são usados para aplicar transformações nas imagens, como redimensionamento, recorte, normalização, etc. Neste caso, estamos utilizando a média e o desvio padrão obtidos na etapa anterior para padronizar as imagens. A padronização é uma técnica comum de pré-processamento de dados que ajuda a acelerar o treinamento e a convergência dos modelos de aprendizado de máquina.</p></li>
    
<li><p style="font-family: 'Times New Roman', Times, serif; font-size: 17px; text-align: justify;"><b>Normalização:</b> A normalização é realizada usando a seguinte fórmula:</p>
<p>$$x^{'} = \frac{x - \bar{x}}{\sigma}$$</p></li>

<li><p style="font-family: 'Times New Roman', Times, serif; font-size: 17px; text-align: justify;"><b>Criação dos DataLoaders:</b> Finalmente, criamos os DataLoaders com o conjunto de dados CIFAR-10. Os DataLoaders são usados para carregar os dados em lotes durante o treinamento do modelo. Eles também podem embaralhar os dados e aplicar transformações. Neste caso, estamos utilizando o DataLoader para carregar as imagens do CIFAR-10 que foram padronizadas na etapa anterior.</p></li>
</ol>

In [ ]:
class GetDataLoaders(object):

    # Método construtor
    def __init__(self):
        self.BATCH_SIZE  = 100 if torch.cuda.is_available() else 64
        self.NUM_WORKERS = int(os.cpu_count()/2)

    # Obtendo a média e desvio padrão das imagens
    def get_mean_and_std(self):

        # Carregando imagens de treino
        trainset   = CIFAR100(root="./data", train=True, transform=ToTensor(), download=True)
        train_data = DataLoader(dataset=trainset, batch_size=self.BATCH_SIZE, shuffle=True, num_workers=self.NUM_WORKERS)

        # Obtendo o número de canais de cores das imagens
        n_channels = next(iter(train_data))[0].size(1)

        # Criando os tensores que armazenarão as médias e desvios padrão
        mean, std = torch.zeros(n_channels), torch.zeros(n_channels)

        # Obtendo a média e desvio padrão das imagens
        for inputs, targets in train_data:
            for i in range(n_channels):
                mean[i] += inputs[:, i, :, :].mean()
                std[i]  += inputs[:, i, :, :].std()
        mean.div_(len(train_data))
        std.div_(len(train_data))

        # Retornando os valores de média e desvio padrão
        return mean, std

    # Obtendo transformadores
    def get_transformers(self):

        # Obtendo média e desvio padrão das imagens
        mean, std = self.get_mean_and_std()

        # Transformações para dados de treino
        train_transform = Compose([
            Resize((224, 224)),
            RandomHorizontalFlip(),
            RandomRotation(15),
            ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
            ToTensor(),
            Normalize(mean, std)
        ])

        # Transformações para dados de validação
        valid_transform = Compose([
            Resize((224, 224)),
            ToTensor(),
            Normalize(mean, std)
        ])

        # Retornando os transformadores
        return train_transform, valid_transform

    # Obtendo os DataLoaders de treino e validação
    def get_dataloaders(self):

        # Obtendo os transformadores
        train_transform, valid_transform = self.get_transformers()

        # Carregando imagens de treino
        trainset   = CIFAR100(root="./data", train=True, transform=train_transform, download=True)
        train_data = DataLoader(dataset=trainset, batch_size=self.BATCH_SIZE, shuffle=True, num_workers=self.NUM_WORKERS)

        # Carregando imagens de validação
        validset   = CIFAR100(root="./data", train=False, transform=valid_transform, download=True)
        valid_data = DataLoader(dataset=validset, batch_size=self.BATCH_SIZE, shuffle=True, num_workers=self.NUM_WORKERS)

        # Retornando os DataLoaders
        return train_data, valid_data

# Obtendo os DataLoaders
train_data, valid_data = GetDataLoaders().get_dataloaders()

100%|██████████| 169001437/169001437 [00:02<00:00, 78151465.33it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data


## 3. DenseNet121 Pré-treinado

### 3.1 Carrega o DenseNet e Define os Processos de Treinamento

In [ ]:
def load_densenet(pretrained=True, fine_tune=True, in_features=1024, out_features=100, use_gpu=True):
    """
    Load and configure a DenseNet model for fine-tuning.

    Args:
        pretrained (bool): If True, load a pre-trained model. Default is True.
        fine_tune (bool): If True, all model parameters are set for training. If False, only the last layer is trained.
        in_features (int): Number of input features for the last layer. Default is 1024.
        out_features (int): Number of output features (classes) for the last layer. Default is 10.
        use_gpu (bool): If True and a GPU is available, the model will be moved to GPU. Default is True.

    Returns:
        torch.nn.Module: A DenseNet model configured for fine-tuning.
    """

    # Load the pre-trained DenseNet model
    model = densenet121(pretrained=pretrained)

    # Enable fine-tuning
    for param in model.parameters():
        param.requires_grad = fine_tune

    # Validate if the in_features matches DenseNet's last layer output
    if in_features != model.classifier.in_features:
        raise ValueError(f"in_features must be equal to {model.classifier.in_features}")

    # Add a new last layer
    model.classifier = nn.Linear(in_features=in_features, out_features=out_features, bias=True)

    # Move the model to GPU if available and desired
    if use_gpu and torch.cuda.is_available():
        model.cuda()

    return model

In [ ]:
class DenseNet121(LightningModule):
    def __init__(self, num_classes=100):
        super().__init__()
        self.model    = load_densenet()  # Certifique-se de que esta função esteja definida corretamente
        self.accuracy = Accuracy(task="multiclass", num_classes=num_classes).to(device)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc = self.accuracy(logits, y)
        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=False)
        self.log("train_acc", acc, on_step=False, on_epoch=True, prog_bar=True, logger=False)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc = self.accuracy(logits, y)
        self.log("test_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=False)
        self.log("test_acc", acc, on_step=False, on_epoch=True, prog_bar=True, logger=False)

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc = self.accuracy(logits, y)
        self.log("valid_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=False)
        self.log("valid_acc", acc, on_step=False, on_epoch=True, prog_bar=True, logger=False)

    def configure_optimizers(self):
        optimizer = optim.AdamW(params=self.parameters(), lr=0.001, weight_decay=0.01)
        lr_scheduler = optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=10, gamma=0.5)
        return {"optimizer": optimizer, "lr_scheduler": {"scheduler": lr_scheduler, "interval": "epoch"}}

# Uso da classe (ajuste 'device' conforme necessário)
modelo = DenseNet121(num_classes=100).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

### 3.2 Configurações de Treinamento

In [ ]:
# Verificando e criando o diretório para os checkpoints
checkpoint_dir = 'best/'
os.makedirs(checkpoint_dir, exist_ok=True)

# Configuração do callback de checkpoint
checkpoint_callback = ModelCheckpoint(
    dirpath=checkpoint_dir,
    monitor='valid_acc',  # Verifique se esta métrica é logada em seu modelo
    mode='max',
    filename='best-model',
    save_top_k=1,
    verbose=True,
)

# Verificando e criando o diretório para os logs
log_dir = "./logs"
os.makedirs(log_dir, exist_ok=True)

# Configuração do treinador
trainer = pl.Trainer(
    accelerator="auto",
    devices=1 if torch.cuda.is_available() else None,
    max_epochs=100,
    logger=CSVLogger(save_dir=log_dir),
    callbacks=[
        EarlyStopping(monitor="valid_acc", patience=3, mode="max"),  # Parada antecipada para evitar overfitting
        checkpoint_callback  # Salva o melhor modelo baseado na métrica 'valid_acc'
    ]
)

### 3.3 Realiza o Treinamento

In [10]:
# Realizando o treinamento
trainer.fit(modelo, train_dataloaders=train_data, val_dataloaders=valid_data)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name     | Type               | Params
------------------------------------------------
0 | model    | DenseNet           | 7.1 M 
1 | accuracy | MulticlassAccuracy | 0     
------------------------------------------------
7.1 M     Trainable params
0         Non-trainable params
7.1 M     Total params
28.225    Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name     | Type               | Params
------------------------------------------------
0 | model    | DenseNet           | 7.1 M 
1 | accuracy | MulticlassAccuracy | 0     
------------------------------------------------
7.1 M     Trainable params
0         Non-trainable params
7.1 M     Total params
28.225    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 500: 'valid_acc' reached 0.60310 (best 0.60310), saving model to '/content/best/best-model.ckpt' as top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 0, global step 500: 'valid_acc' reached 0.60310 (best 0.60310), saving model to '/content/best/best-model.ckpt' as top 1


...

INFO: Epoch 11, global step 6000: 'valid_acc' was not in top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 11, global step 6000: 'valid_acc' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 12, global step 6500: 'valid_acc' was not in top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 12, global step 6500: 'valid_acc' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 13, global step 7000: 'valid_acc' was not in top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 13, global step 7000: 'valid_acc' was not in top 1


## 4. Avaliação do Modelo DenseNet

In [12]:
# Carregando os pesos do modelo
checkpoint = torch.load("/content/best/best-model.ckpt")
modelo.load_state_dict(checkpoint['state_dict'])

# Rsultados para dados de validação
trainer.test(modelo, valid_data)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

[{}]